In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
client_data = pd.read_csv('./data/clients.csv')
student_data = pd.read_csv('./data/fyp_students_datasets.csv')

In [3]:
client_data.head(3)

,Company Name,Title,Project brief,Project Background,Requiements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...


In [4]:
student_data.head(4)

,Group No.,Unnamed: 1,Title,Supervsior,Met with supervisor?,Preferred name on the CASUGOL certificate,FULL NAME As your IC/Passport,CV sent\n(Y/N),Program,Areas of Interest and your technical strength,...,Leader / Member,Email ADD,SD EMAIL,Contact No,Semester,Check Study Plan: Capstone 2 is it JAN-FEB,CGPA,GPA,Industry - Contact Point,Company
0,1.0,1.0,A Trust Index Model to Detect Fake News in Soc...,Mr. Tee Wee Jing,Yes,Nicholas Ryan Farm,Nicholas Ryan Farm,Yes,Bachelor of Computer Science,"Full Stack Web Development, Project Management...",...,Leader,nicholasfarm27@gmail.com,nicholasryan.farm@sd.taylors.edu.my,019-4549277,Sem 4,Yes,3.94,3.50,NaN,ProximaX
1,NaN,2.0,NaN,NaN,NaN,Gan Zheng Jie,Gan Zheng Jie,Yes,Bachelor of Software Engineering,"Full stack mobile application development, Dat...",...,Member,ganzj45@gmail.com,zhengjie.gan@sd.taylors.edu.my,017-5015966,Sem 5,Yes,3.85,3.78,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,Low Yee Yieng,Low Yee Yieng,Yes,Bachelor of Software Engineering,"IT Analysis, Web/Mobile App Development, UI/UX...",...,Member,lyeeyieng@gmail.com,yeeyieng.low@sd.taylors.edu.my,010-2776323,Sem 5,Yes,3.87,3.75,NaN,NaN
3,NaN,4.0,NaN,NaN,NaN,Sachin Vijaya Kumar,Saachin Vijaya Kumar,Yes,Bachelor of Computer Science,"Front-end development, UI/UX Design, Data Anal...",...,Member,taylorssvk@gmail.com,saachin.vijayakumar@sd.taylors.edu.my,018-3137551,Sem 5,Yes,3.59,3.67,NaN,NaN


# Exploratory Data Analysis

In [5]:
client_data['Requirements'] = client_data['Project brief '].map(str) + ' ' + client_data['Project Background']+' '+client_data['Requiements']

In [6]:
client_data.head(3)


,Company Name,Title,Project brief,Project Background,Requiements,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...,Automated testing suite for all software produ...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...,Identify the effective indicators of obesity b...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...,To improve employee communications and engagem...


In [7]:
client_data = client_data.drop(['Requiements'],axis=1)

In [8]:
client_data = client_data.fillna(' ')

In [9]:
client_data.head()

,Company Name,Title,Project brief,Project Background,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Automated testing suite for all software produ...
1,,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,Identify the effective indicators of obesity b...
2,,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,To improve employee communications and engagem...
3,PurposePurchase Sdh Bhd,PurposePurchase,"\ncause-related ecommerce app, as well as a pl...",PurposePurchase is an online e-commerce platfo...,"\ncause-related ecommerce app, as well as a pl..."
4,,Educational Game,The aim of this project is to create an educat...,"The students will be given a concept document,...",The aim of this project is to create an educat...


In [10]:
client_data.shape

(25, 5)

In [11]:
stopwords_ = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [12]:
# Create word tokens
def token_txt(token):
    return token not in stopword_ and token not in list(string.punctuation) and len(token) > 2   
  

In [13]:
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "", text)
  text = re.sub("(\\d|\\W)+", " ", text) 
  text = text.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if token_txt(word)]
  clean_text2 = [word for word in clean_text if token_txt(word)]
  return " ".join(clean_text2)

In [14]:
client_data['Requirements'] = client_data['Requirements'].apply(clean_txt)

In [15]:
client_data.tail()

,Company Name,Title,Project brief,Project Background,Requirements
20,,,,,
21,Agensi Pekerjaan & Latihan TAPRO Sdn Bhd,(CRM) tool for recruitment.,"Currently, selected information of the candida...",Aim to build a CRM tool for recruitment,currently select information candidates extrac...
22,People Systems Consultancy Sdn Bhd,App to build Financial Inclusion and Intellige...,Build the financial resilience and intelligenc...,seeking support and assistance to: \n(a) build...,build financial resilience intelligence differ...
23,,Statistics: Zero to Hundred,"In the world of education, e-learning platform...",The aim of this project is to create an e-lear...,world education learn platform essential make ...
24,,WhatsApp Messenger Forensic Analysis Based on ...,,,


## Student corpus

In [16]:
student_data['Requirements'] = student_data['Program '].map(str) + ' ' + student_data['Areas of Interest and your technical strength']

In [17]:
student_data = student_data[['FULL NAME As your IC/Passport','Requirements']]

In [18]:
student_data.head(3)

,FULL NAME As your IC/Passport,Requirements
0,Nicholas Ryan Farm,Bachelor of Computer Science Full Stack Web De...
1,Gan Zheng Jie,Bachelor of Software Engineering Full stack m...
2,Low Yee Yieng,"Bachelor of Software Engineering IT Analysis,..."


In [19]:
student_data['Requirements'] = student_data['Requirements'].apply(clean_txt)